Unduh data korpus Wikipedia Bahasa Indonesia melalui link berikut https://dumps.wikimedia.org/idwiki/latest/. Cari file dengan nama “idwiki-latest-pages-articles.xml.bz2”. Data korpus setiap waktunya mengalami peningkatan. Sekitar agustus 2019 terdapat 406855 kosa kata dengan ukuran 512 MB.

Data korpus tersebut masih menggunakan format XML sehingga perlu di olah ke dalam bentuk teks. library Gensim sudah menyediakan teknik pra poses ini. Untuk mengkonversi format tersebut dapat menggunakan kode berikut:

In [1]:
from __future__ import print_function
 
# Ignore warnings dari gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
 
import logging
import os.path
import sys
 
from gensim.corpora import WikiCorpus
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "input/idwiki-latest-pages-articles.xml.bz2"
namaFileOutput = "output/wiki.id.text"
 
space = " "
i = 0
 
# Write file ke variabel namaFileOutput encoder utf-8
output = open(namaFileOutput, 'w', encoding='utf-8')
 
# lower=False: huruf kecil dan besar dibedakan
wiki = WikiCorpus(namaFileInput, lemmatize=None, dictionary={}, lower=False)
for text in wiki.get_texts():
    output.write(' '.join(text) + '\n')
    i = i + 1
    if i % 10000 == 0:
        logger.info("Saved " + str(i) + " articles")
 
output.close()
logger.info("Finished Saved " + str(i) + " articles")

2023-12-06 00:18:48,475: INFO: running C:\Users\mmahf\miniconda3\envs\latihanstki\lib\site-packages\ipykernel_launcher.py -f C:\Users\mmahf\AppData\Roaming\jupyter\runtime\kernel-19566f0c-3349-49f0-805b-c5f073a7deba.json
2023-12-06 00:19:47,981: INFO: Saved 10000 articles
2023-12-06 00:20:30,603: INFO: Saved 20000 articles
2023-12-06 00:21:05,895: INFO: Saved 30000 articles
2023-12-06 00:21:37,445: INFO: Saved 40000 articles
2023-12-06 00:22:11,540: INFO: Saved 50000 articles
2023-12-06 00:22:43,207: INFO: Saved 60000 articles
2023-12-06 00:23:14,996: INFO: Saved 70000 articles
2023-12-06 00:23:48,709: INFO: Saved 80000 articles
2023-12-06 00:24:38,596: INFO: Saved 90000 articles
2023-12-06 00:25:14,079: INFO: Saved 100000 articles
2023-12-06 00:25:46,252: INFO: Saved 110000 articles
2023-12-06 00:26:15,232: INFO: Saved 120000 articles
2023-12-06 00:26:56,676: INFO: Saved 130000 articles
2023-12-06 00:27:29,425: INFO: Saved 140000 articles
2023-12-06 00:27:51,620: INFO: Saved 150000 ar

Dari kode di atas akan menghasilkan output file dari rangkaian beberapa artikel, di mana satu barisnya mewakili satu artikel.

Setelah data berhasil terkonversi, selanjutnya dilakukan training Word2Vec menggunakan kode berikut:

In [2]:
import multiprocessing
import logging
import os.path
import sys
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "output/wiki.id.text"
namaFileOutput = "output/w2vec_wiki_id300_0.txt"
 
 
# size 300 = 300 dimensi vektor, window 10 = 10 pengaruh kata disekitarnya, min_count 5 = kata-kata yang muncul < 5 kali akan dikeluarkan dari kosakata dan diabaikan selama pelatihan, sg 0 = cbow / sg 1 = skip gram 
model = Word2Vec(LineSentence(namaFileInput), vector_size=300, window=10, min_count=5, sg=0, workers=multiprocessing.cpu_count())
 
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)
model.wv.save_word2vec_format(namaFileOutput, binary=False)

2023-12-06 00:42:40,930: INFO: running C:\Users\mmahf\miniconda3\envs\latihanstki\lib\site-packages\ipykernel_launcher.py -f C:\Users\mmahf\AppData\Roaming\jupyter\runtime\kernel-19566f0c-3349-49f0-805b-c5f073a7deba.json
2023-12-06 00:42:40,932: INFO: collecting all words and their counts
2023-12-06 00:42:40,949: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-12-06 00:42:45,283: INFO: PROGRESS: at sentence #10000, processed 10738098 words, keeping 353874 word types
2023-12-06 00:42:48,238: INFO: PROGRESS: at sentence #20000, processed 17334952 words, keeping 490368 word types
2023-12-06 00:42:50,334: INFO: PROGRESS: at sentence #30000, processed 22148080 words, keeping 591133 word types
2023-12-06 00:42:52,181: INFO: PROGRESS: at sentence #40000, processed 26549341 words, keeping 676278 word types
2023-12-06 00:42:54,229: INFO: PROGRESS: at sentence #50000, processed 31270076 words, keeping 775169 word types
2023-12-06 00:42:55,931: INFO: PROGRESS: at sent

2023-12-06 00:44:03,646: INFO: EPOCH 0 - PROGRESS: at 0.09% examples, 763312 words/s, in_qsize 1, out_qsize 0
2023-12-06 00:44:04,667: INFO: EPOCH 0 - PROGRESS: at 0.23% examples, 806020 words/s, in_qsize 4, out_qsize 1
2023-12-06 00:44:05,682: INFO: EPOCH 0 - PROGRESS: at 0.34% examples, 829679 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:44:06,683: INFO: EPOCH 0 - PROGRESS: at 0.52% examples, 841967 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:44:07,683: INFO: EPOCH 0 - PROGRESS: at 0.72% examples, 842219 words/s, in_qsize 1, out_qsize 0
2023-12-06 00:44:08,683: INFO: EPOCH 0 - PROGRESS: at 0.87% examples, 845503 words/s, in_qsize 0, out_qsize 1
2023-12-06 00:44:09,699: INFO: EPOCH 0 - PROGRESS: at 1.05% examples, 847996 words/s, in_qsize 6, out_qsize 0
2023-12-06 00:44:10,718: INFO: EPOCH 0 - PROGRESS: at 1.29% examples, 851269 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:44:11,718: INFO: EPOCH 0 - PROGRESS: at 1.49% examples, 854962 words/s, in_qsize 0, out_qsize 0
2023-12-06

2023-12-06 00:45:19,401: INFO: EPOCH 0 - PROGRESS: at 31.41% examples, 782131 words/s, in_qsize 15, out_qsize 0
2023-12-06 00:45:20,404: INFO: EPOCH 0 - PROGRESS: at 34.39% examples, 783919 words/s, in_qsize 11, out_qsize 1
2023-12-06 00:45:21,406: INFO: EPOCH 0 - PROGRESS: at 37.38% examples, 785299 words/s, in_qsize 14, out_qsize 0
2023-12-06 00:45:22,407: INFO: EPOCH 0 - PROGRESS: at 40.26% examples, 786069 words/s, in_qsize 15, out_qsize 0
2023-12-06 00:45:23,410: INFO: EPOCH 0 - PROGRESS: at 43.11% examples, 786780 words/s, in_qsize 15, out_qsize 0
2023-12-06 00:45:24,435: INFO: EPOCH 0 - PROGRESS: at 46.52% examples, 787352 words/s, in_qsize 15, out_qsize 0
2023-12-06 00:45:25,436: INFO: EPOCH 0 - PROGRESS: at 49.89% examples, 787843 words/s, in_qsize 12, out_qsize 0
2023-12-06 00:45:26,442: INFO: EPOCH 0 - PROGRESS: at 50.67% examples, 787449 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:45:27,457: INFO: EPOCH 0 - PROGRESS: at 51.10% examples, 786817 words/s, in_qsize 0, out_qs

2023-12-06 00:46:34,204: INFO: EPOCH 0 - PROGRESS: at 87.83% examples, 769862 words/s, in_qsize 0, out_qsize 2
2023-12-06 00:46:35,215: INFO: EPOCH 0 - PROGRESS: at 88.43% examples, 769393 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:46:36,219: INFO: EPOCH 0 - PROGRESS: at 88.97% examples, 768690 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:46:37,228: INFO: EPOCH 0 - PROGRESS: at 89.54% examples, 768246 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:46:38,233: INFO: EPOCH 0 - PROGRESS: at 90.20% examples, 768275 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:46:39,238: INFO: EPOCH 0 - PROGRESS: at 90.86% examples, 768172 words/s, in_qsize 0, out_qsize 1
2023-12-06 00:46:40,256: INFO: EPOCH 0 - PROGRESS: at 91.47% examples, 768094 words/s, in_qsize 0, out_qsize 1
2023-12-06 00:46:41,286: INFO: EPOCH 0 - PROGRESS: at 92.04% examples, 768039 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:46:42,309: INFO: EPOCH 0 - PROGRESS: at 92.51% examples, 768242 words/s, in_qsize 0, out_qsize 0
2

2023-12-06 00:47:48,811: INFO: EPOCH 1 - PROGRESS: at 17.52% examples, 776888 words/s, in_qsize 3, out_qsize 0
2023-12-06 00:47:49,815: INFO: EPOCH 1 - PROGRESS: at 18.12% examples, 777177 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:47:50,816: INFO: EPOCH 1 - PROGRESS: at 18.69% examples, 777500 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:47:51,827: INFO: EPOCH 1 - PROGRESS: at 19.29% examples, 777654 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:47:52,833: INFO: EPOCH 1 - PROGRESS: at 19.82% examples, 778089 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:47:53,839: INFO: EPOCH 1 - PROGRESS: at 20.27% examples, 778332 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:47:54,854: INFO: EPOCH 1 - PROGRESS: at 20.77% examples, 778463 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:47:55,860: INFO: EPOCH 1 - PROGRESS: at 21.23% examples, 778334 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:47:56,864: INFO: EPOCH 1 - PROGRESS: at 21.71% examples, 777758 words/s, in_qsize 3, out_qsize 0
2

...

2023-12-06 00:52:47,049: INFO: EPOCH 3 - PROGRESS: at 1.07% examples, 852062 words/s, in_qsize 1, out_qsize 0
2023-12-06 00:52:48,063: INFO: EPOCH 3 - PROGRESS: at 1.28% examples, 840952 words/s, in_qsize 5, out_qsize 2
2023-12-06 00:52:49,066: INFO: EPOCH 3 - PROGRESS: at 1.48% examples, 840376 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:52:50,066: INFO: EPOCH 3 - PROGRESS: at 1.69% examples, 838798 words/s, in_qsize 1, out_qsize 0
2023-12-06 00:52:51,080: INFO: EPOCH 3 - PROGRESS: at 1.94% examples, 838998 words/s, in_qsize 1, out_qsize 0
2023-12-06 00:52:52,103: INFO: EPOCH 3 - PROGRESS: at 2.16% examples, 840056 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:52:53,114: INFO: EPOCH 3 - PROGRESS: at 2.45% examples, 844798 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:52:54,114: INFO: EPOCH 3 - PROGRESS: at 2.72% examples, 845003 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:52:55,125: INFO: EPOCH 3 - PROGRESS: at 3.03% examples, 843905 words/s, in_qsize 0, out_qsize 0
2023-12-06

2023-12-06 00:54:01,743: INFO: EPOCH 3 - PROGRESS: at 51.56% examples, 823569 words/s, in_qsize 0, out_qsize 1
2023-12-06 00:54:02,751: INFO: EPOCH 3 - PROGRESS: at 52.16% examples, 824330 words/s, in_qsize 0, out_qsize 1
2023-12-06 00:54:03,759: INFO: EPOCH 3 - PROGRESS: at 53.01% examples, 824292 words/s, in_qsize 1, out_qsize 1
2023-12-06 00:54:04,761: INFO: EPOCH 3 - PROGRESS: at 53.83% examples, 824707 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:54:05,765: INFO: EPOCH 3 - PROGRESS: at 54.54% examples, 825028 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:54:06,784: INFO: EPOCH 3 - PROGRESS: at 55.11% examples, 824637 words/s, in_qsize 0, out_qsize 1
2023-12-06 00:54:07,793: INFO: EPOCH 3 - PROGRESS: at 55.60% examples, 824607 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:54:08,795: INFO: EPOCH 3 - PROGRESS: at 56.10% examples, 825066 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:54:09,804: INFO: EPOCH 3 - PROGRESS: at 56.60% examples, 825116 words/s, in_qsize 0, out_qsize 1
2

2023-12-06 00:55:16,410: INFO: EPOCH 3 - PROGRESS: at 96.28% examples, 822695 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:55:17,414: INFO: EPOCH 3 - PROGRESS: at 96.88% examples, 822769 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:55:18,439: INFO: EPOCH 3 - PROGRESS: at 97.53% examples, 822702 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:55:19,457: INFO: EPOCH 3 - PROGRESS: at 98.09% examples, 822736 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:55:20,463: INFO: EPOCH 3 - PROGRESS: at 98.67% examples, 823042 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:55:21,467: INFO: EPOCH 3 - PROGRESS: at 99.50% examples, 823249 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:55:22,267: INFO: EPOCH 3: training on 149788691 raw words (133570960 effective words) took 162.3s, 822917 effective words/s
2023-12-06 00:55:23,301: INFO: EPOCH 4 - PROGRESS: at 0.10% examples, 822167 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:55:24,301: INFO: EPOCH 4 - PROGRESS: at 0.24% examples, 843292 words/s, 

2023-12-06 00:56:31,014: INFO: EPOCH 4 - PROGRESS: at 27.61% examples, 818616 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:56:32,025: INFO: EPOCH 4 - PROGRESS: at 28.05% examples, 818746 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:56:33,025: INFO: EPOCH 4 - PROGRESS: at 28.57% examples, 819082 words/s, in_qsize 0, out_qsize 1
2023-12-06 00:56:34,035: INFO: EPOCH 4 - PROGRESS: at 29.06% examples, 818698 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:56:35,041: INFO: EPOCH 4 - PROGRESS: at 30.20% examples, 819896 words/s, in_qsize 6, out_qsize 1
2023-12-06 00:56:36,045: INFO: EPOCH 4 - PROGRESS: at 33.21% examples, 821395 words/s, in_qsize 10, out_qsize 0
2023-12-06 00:56:37,051: INFO: EPOCH 4 - PROGRESS: at 36.31% examples, 822523 words/s, in_qsize 13, out_qsize 0
2023-12-06 00:56:38,067: INFO: EPOCH 4 - PROGRESS: at 39.40% examples, 823966 words/s, in_qsize 4, out_qsize 0
2023-12-06 00:56:39,076: INFO: EPOCH 4 - PROGRESS: at 42.58% examples, 825004 words/s, in_qsize 12, out_qsize 

2023-12-06 00:57:45,645: INFO: EPOCH 4 - PROGRESS: at 87.35% examples, 809504 words/s, in_qsize 13, out_qsize 2
2023-12-06 00:57:46,647: INFO: EPOCH 4 - PROGRESS: at 87.90% examples, 808777 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:57:47,655: INFO: EPOCH 4 - PROGRESS: at 88.49% examples, 808104 words/s, in_qsize 1, out_qsize 0
2023-12-06 00:57:48,663: INFO: EPOCH 4 - PROGRESS: at 89.07% examples, 807304 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:57:49,664: INFO: EPOCH 4 - PROGRESS: at 89.65% examples, 806457 words/s, in_qsize 0, out_qsize 1
2023-12-06 00:57:50,669: INFO: EPOCH 4 - PROGRESS: at 90.26% examples, 805979 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:57:51,681: INFO: EPOCH 4 - PROGRESS: at 90.89% examples, 805346 words/s, in_qsize 1, out_qsize 1
2023-12-06 00:57:52,695: INFO: EPOCH 4 - PROGRESS: at 91.44% examples, 804586 words/s, in_qsize 0, out_qsize 0
2023-12-06 00:57:53,696: INFO: EPOCH 4 - PROGRESS: at 91.99% examples, 803979 words/s, in_qsize 0, out_qsize 0


Terdapat beberapa parameter diantaranya:

<b>vector_size</b>: Ukuran dimensi vector yang mewakili setiap token atau kata. Jika memiliki data yang terbatas, nilai size yang digunakan sebaiknya lebih kecil karena akan mempengaruhi kata unik di sekitarnya. Namun, jika memiliki banyak data maka dapat bereksperimen dengan berbagai ukuran.

<b>window</b>: Jarak maksimum antara kata target dengan kata di sekitarnya. Jika nilai window yang digunakan besar, maka terdapat banyak kata terkait disekitarnya (baik di posisi kiri dan kanan dari kata konteks). Secara teori, jika window lebih kecil akan memberikan istilah yang lebih spesifik terhadap suatu konteks kata.

<b>min_count</b>: Frekuensi minimal jumlah kata. Model akan mengabaikan kata-kata yang tidak memenuhi nilai min_count. Kata yang jarang muncul biasanya tidak terlalu penting, jadi lebih baik untuk dihilangkan. Parameter ini mungkin lebih berpengaruh pada effisiensi penggunaan memory dan ukuran file model.

<b>sg</b>: 0 untuk arsitektur Word2Vec CBOW dan 1 untuk arsitektur Word2Vec Skip-gram.

<b>workers</b>: Berapa banyak threads yang digunakan untuk melakukan multiprocessing.

Model yang sudah disimpan dapat dilakukan skenario penggunaannya sebagaimana pada kode di bawah ini untuk melihat kedekatan hubungan antara kata.

In [3]:
import gensim
#import gensim.models.keyedvectors as word2vec


namaFileModel = "w2vec_wiki_id300_0.txt"

#model = gensim.models.Word2Vec.load(namaFileModel)
#model = word2vec.KeyedVectors.load_word2vec_format(namaFileModel, binary=True)

sim = model.wv.most_similar("Bakso")
print("10 kata terdekat dari Bakso:{}".format(sim))
sim = model.wv.most_similar("Mahasiswa")
print("10 kata terdekat dari Mahasiswa:{}".format(sim))
 
sim = model.wv.similarity("Yogyakarta", "Surakarta")
print("Kedekatan Yogyakarta-Surakarta: {}".format(sim))
sim = model.wv.similarity("Yogyakarta", "Semarang")
print("Kedekatan Yogyakarta-Semarang: {}".format(sim))
 
sim = model.wv.most_similar_cosmul(positive=['minuman', 'rendang'], negative=['makanan'])
print("makanan-rendang, minuman-?: {}".format(sim))
sim = model.wv.most_similar_cosmul(positive=['mobil', 'honda'], negative=['motor'])
print("motor-honda, mobil-?: {}".format(sim))

10 kata terdekat dari Bakso:[('Rujak', 0.8280162811279297), ('Bakmi', 0.8200593590736389), ('Pecel', 0.8118868470191956), ('bakso', 0.7965819835662842), ('Kerupuk', 0.7877241373062134), ('Goreng', 0.7716336846351624), ('Sate', 0.7678637504577637), ('Cingur', 0.7609280347824097), ('Serabi', 0.7536329627037048), ('Pindang', 0.7525295615196228)]
10 kata terdekat dari Mahasiswa:[('Mahasiwa', 0.5986729860305786), ('Pelajar', 0.5973383784294128), ('HMI', 0.5957608222961426), ('HMJ', 0.5813392400741577), ('BEM', 0.5781722664833069), ('PMII', 0.5623728632926941), ('Kemahasiswaan', 0.5619442462921143), ('PMKRI', 0.5522595643997192), ('IKA', 0.5512791872024536), ('kemahasiswaan', 0.5498942732810974)]
Kedekatan Yogyakarta-Surakarta: 0.7878975868225098
Kedekatan Yogyakarta-Semarang: 0.650667667388916
makanan-rendang, minuman-?: [('kola', 0.8548056975850563), ('martabak', 0.8518867489868422), ('Fanta', 0.851636415920935), ('panganan', 0.8461340103685069), ('Minuman', 0.8461154699239732), ('lemon', 